# Mean Reciprocal Rank (MRR)を実装
MRRは以下のように定式化される．セッションの番号を表すラベルを$j$とし，計$Q$個のセッションがあるものとする．\
また$j$番目のセッションで購入された商品のラベルを$t_j$とする．\
いま，$j$番目のセッションに対する商品$t_j$のレコメンド順位が${\rm rank}_j$であったとき，MRRは
$$
{\rm MRR} = \dfrac{1}{|Q|} \sum_{j=1}^{|Q|} \dfrac{1}{{\rm rank}_j}
$$
のように定義される．ただし，商品$t_j$がレコメンドに入っていない場合は${\rm rank}_j \to \infty$とみなし，$\dfrac{1}{{\rm rank}_j} = 0$として計算する．

# セットアップ
ほとんど使わない

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import re
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# MRRを計算する関数の定義

In [8]:
# df_targetは正解ラベル，列には["session_id", "item_id"]が必要．
# df_predはsubmitと同様の順位表，列には["session_id", "item", "rank"]が必要．
def mean_reciprocal_rank(df_target, df_pred):
    df_MRR = pd.merge(df_train_purchases,df_pred, hoow = "left", on = ["session_id", "item_id"]) # 各セッションごとに，購入された商品の順位を記録
    df_MRR["reciprocal_rank"] = 1.0 / df_MRR["rank"] # reciprocal rankを計算
    df_MRR["reciprocal_rank"].fillna(0.0, inplace = True) # 順位がない場合はreciprocal rankは0になる
    return df_MRR["reciprocal_rank"].mean() # 平均値がMRRになる

# 簡単な例で出力を確認

## 正解ラベルの読み込み

In [3]:
num_sample = 10000 # 今回使用するサンプル数

# 正解ラベルの読み込み
path_data = r'C:\Users\yota-\Desktop\study\mystudy\recsys\dressipi_recsys2022\data'
df_train_purchases = pd.read_csv(path_data + r"\train_purchases.csv").sample(n = num_sample) # 一部だけをサンプル
display(df_train_purchases)

,session_id,item_id,date
793585,3523961,8858,2021-05-22 20:56:10.178
18485,82374,7412,2020-08-28 06:40:21.225
244272,1087414,14873,2020-09-05 18:22:50.295
653557,2901529,21781,2021-03-01 15:32:05.481
176858,787896,19087,2021-03-22 22:21:52.239
...,...,...,...
527075,2339999,16017,2020-05-12 06:57:37.568
970439,4308921,26909,2020-09-10 08:26:55.552
221022,985059,6180,2021-01-26 19:51:10.593
766647,3404322,25580,2021-04-25 13:54:51.151


## 入力データから単純な出現回数の順位表を作る

In [4]:
# 入力データの読み込み
df_train_sessions = pd.read_csv(path_data + r"\train_sessions.csv")

# 入力データ中の出現回数のみから順位表を作る
num_pred = 100
df_rank = df_train_purchases.groupby("item_id").size().to_frame().rename(columns = {0:"count"})
df_rank = df_rank.sort_values(by = "count", ascending = False).head(num_pred)

df_rank["item_id"] = df_rank.index.values
df_rank["rank"] = range(1, num_pred+1, 1)
df_rank.drop(axis = 1, columns = ["count"], inplace = True)
display(df_rank)

,item_id,rank
item_id,,
8060,8060,1
26853,26853,2
2447,2447,3
19882,19882,4
8622,8622,5
...,...,...
20028,20028,96
5691,5691,97
12464,12464,98


## 単純な順位表のみから予測結果を作る

In [6]:
# pandasはcross_joinがないらしい(マジ？)
def cross_join(df_a, df_b, common_key=None):
    # 同じ列名を持つ場合にはうまく動かないので注意
    df_a['tmp'] = 1
    df_b['tmp'] = 1
    return pd.merge(df_a, df_b, how='outer').drop("tmp", axis = 1)

# 順位表に基づいてpredictionする関数を作成
def predict_by_rank_table(df_test):
    session_id_list = df_test["session_id"].unique()
    df_pred = pd.DataFrame(columns = ["session_id"])
    df_pred["session_id"] = session_id_list
    return cross_join(df_pred, df_rank)

df_pred = predict_by_rank_table(df_train_purchases)
display(df_pred)

,session_id,item_id,rank
0,3523961,8060,1
1,3523961,26853,2
2,3523961,2447,3
3,3523961,19882,4
4,3523961,8622,5
...,...,...,...
999995,1339966,20028,96
999996,1339966,5691,97
999997,1339966,12464,98
999998,1339966,24315,99


順位表をそのまま出力した場合のスコアが0.16程度だったので，割とあっていそう．何度か試しても大きなずれはない．

In [9]:
# df_train_purchasesが正解ラベル，df_predが予測の順位表
mean_reciprocal_rank(df_train_purchases, df_pred)

TypeError: merge() missing 1 required positional argument: 'right'